In [4]:
#This is the code from the presentation
import numpy as np


class fsa:
    def __init__(self, initial=1, final=set(), transitions={}):
        """Class for finite-state automata.
        
        Arguments
        ---------
        initial: int or string
            name of initial state
            default: 1
        final: set
            set of final states
            default: empty set
        transitions: dict
            dictionary of the form {current_state: {arc_label: new_state}}
            default: empty dictionary
        """
        self.I = initial
        self.F = final
        self.T = transitions
        
    def mm_con(self):
        """Method to make a FSA work with matrix multiplication
        states = a list of the states
        n = number to states (integer)
        ni = new initial, array
        nf = new final, array
        nt = new transition, dictionary in the form {Tranisition: array}
        returns a tuple in the form (ni, nf, nt, n)
        """
        #states compiles all the states so we can count them- adding the transitions then the final states if not in trans
        states = [item for item in self.T]
        for item in self.F:
            if item not in states:
                states.append(item)
        n = len(states)
        #creates a "blank" array of zeros for ni and nf
        ni = np.zeros( (1, n) )
        nf = np.zeros( (n, 1) )
        #sets the column that corresponds to inital state to 1
        ni[0][self.I-1] = 1
        #create a new dictionary for the transition arrays
        nt = {}
        #creates the array for final by replacing the 0 with a 1 in the correct place
        for num in self.F:
            nf[num-1][0] = 1
        #creates the arrays for transitions
        #creates a new word "blank" array then sets the correct value to 1
        #if an array for that word already exists they are added
        for item in self.T:
            cs = self.T[item]
            for word in cs:
                nw = np.zeros( (n, n) )
                nw[item-1][cs[word]-1] = 1
                if np.any(nt.get(word)):
                    nt[word] = nt[word] + nw
                else:
                    nt[word] = nw
        #returns a tuple of all the values needed for accepts
        return ni, nf, nt, n
        
    def n_accepts(self, sentence):
        """a new accept method
        sentence in the form of a tokenized list (like original accept method)
        conversion runs the convert method
        cs = current state, set to the initial array
        nt, nf, n = setting these valoue to their corresponding values in the returned tuple
        """
        conversion = self.mm_con()
        cs = conversion[0]
        nt = conversion[2]
        nf = conversion[1]
        n = conversion[3]
        #multiply the inital by the first transition, then by the next transition
        #if a value is above 1 then it resets it to 1 so the values do not grow
        for word in sentence:
            cs = cs.dot(nt.get(word))
            for j in range(n):
                item = cs[0][j]
                if item > 1:
                    item = 1
        #multiply the current state by the final
        cs = cs.dot(nf)
        return True if cs >= [1] else False

In [12]:
#for non-deterministic FSA not including empty strings
#this also improves the "normalizing" instead of using for loops it uses a "sign" method
#changes how the fsa is input as well
import numpy as np


class fsa:
    def __init__(self, initial={1}, final=set(), transitions={}):
        """Class for finite-state automata.
        
        Arguments
        ---------
        initial: int or string
            name of initial state
            default: {1}
        final: set
            set of final states
            default: empty set
        transitions: dict
            dictionary of the form {current_state: {arc_label: {new_state}}}
            default: empty dictionary
        """
        self.I = initial
        self.F = final
        self.T = transitions
        
    def mm_con(self):
        """Method to make a FSA work with matrix multiplication
        states = a list of the states
        n = number to states (integer)
        ni = new initial, array
        nf = new final, array
        nt = new transition, dictionary in the form {Tranisition: array}
        returns a tuple in the form (ni, nf, nt, n)
        """
        #states compiles all the states so we can count them- adding the transitions then the final states if not in trans
        states = [item for item in self.T]
        for item in self.F:
            if item not in states:
                states.append(item)
        n = len(states)
        #creates a "blank" array of zeros for ni and nf
        ni = np.zeros( (1, n) )
        nf = np.zeros( (n, 1) )
        #sets the column that corresponds to inital state to 1
        for num in self.I:
            ni[0][num-1] = 1
        #create a new dictionary for the transition arrays
        nt = {}
        #creates the array for final by replacing the 0 with a 1 in the correct place
        for num in self.F:
            nf[num-1][0] = 1
        #creates the arrays for transitions
        #creates a new word "blank" array then sets the correct value to 1
        #basically just added another for loop to "access" the new dictionary
        #if an array for that word already exists they are added
        for item in self.T:
            cs = self.T[item]
            for word in cs:
                css = cs.get(word)
                for numb in css:
                    nw = np.zeros( (n, n) )
                    nw[item-1][numb-1] = 1
                    if np.any(nt.get(word)):
                        nt[word] = nt[word] + nw
                    else:
                        nt[word] = nw
        #returns a tuple of all the values needed for accepts
        return ni, nf, nt, n
        
    def n_accepts(self, sentence):
        """a new accept method
        sentence in the form of a tokenized list (like original accept method)
        conversion runs the convert method
        cs = current state, set to the initial array
        nt, nf, n = setting these valoue to their corresponding values in the returned tuple
        """
        conversion = self.mm_con()
        cs = conversion[0]
        nt = conversion[2]
        nf = conversion[1]
        n = conversion[3]
        #multiply the inital by the first transition, then by the next transition
        #the sign makes it so 0=0 and any pos. number=1
        for word in sentence:
            cs = cs.dot(nt.get(word))
            cs = np.sign(cs)
        #multiply the current state by the final
        cs = cs.dot(nf)
        return True if cs >= [1] else False

In [10]:
#example
fsa1 = fsa(initial=1, final={1,2,4}, transitions={1: {"a": 2, "b": 3, "c": 1}, 2: {"b": 2}, 3: {"a": 3, "c": 4}})
fsa1.mm_con()

(array([[1., 0., 0., 0.]]), array([[1.],
        [1.],
        [0.],
        [1.]]), {'a': array([[0., 1., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 0.]]), 'b': array([[0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]), 'c': array([[1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 0.]])}, 4)

In [13]:
fsa2 = fsa(initial={1}, final={2,3}, transitions={1:{a:{2,3}}, 2:{b:{3,4}}, 3:{b:{3}}, 4:{c:{3}}}
fsa2.mm_con()

SyntaxError: invalid syntax (<ipython-input-13-2a773582b554>, line 2)